In [8]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [9]:
Base = declarative_base()

In [10]:
class Person(Base):
    __tablename__ = "people"
    ssn = Column("ssn", Integer, primary_key=True)
    first_name = Column('first_name', String)
    last_name = Column('last_name', String)
    gender = Column('gender', CHAR)
    age = Column("age", Integer)

    def __init__(self, ssn, first_name, last_name, gender, age) -> None:
        self.ssn = ssn
        self.first_name = first_name
        self.last_name = last_name
        self.gender = gender
        self.age = age


    def __repr__(self) -> str:
        return f'({self.ssn}) ({self.first_name}) ({self.last_name})'


class Thing(Base):
    __tablename__ = "things"
    tid = Column('tid', Integer, primary_key=True)
    description = Column('description', String)
    owner = Column('owner', ForeignKey('people.ssn'))

    def __init__(self, tid, description, owner):
        self.tid = tid
        self.description = description
        self.owner = owner 

In [12]:
engine = create_engine('sqlite:///test3.sqlite')
engine

Engine(sqlite:///test3.sqlite)

In [13]:
Base.metadata.create_all(bind=engine)

In [15]:
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
person = Person(1, 'Mike', 'Smith', 'm', 36)
session.add(person)
session.commit()

In [35]:
p1 = Person(2, 'Anna', 'Smith', 'm', 25)
p2 = Person(3, 'Scott', 'Dylan', 'm', 18)
p3 = Person(4, 'Cassie', 'Ego', 'm', 20)

session.add_all([p1,p2,p3])
session.commit()


In [16]:
t1 = Thing(1, 'Book', 1)
t2 = Thing(2, 'Bike', 2)
session.add_all([t1, t2])
session.commit()

### QUERY

In [24]:
result = session.query(Person).all()
result

[(1) (Mike) (Smith),
 (2) (Anna) (Smith),
 (3) (Scott) (Dylan),
 (4) (Cassie) (Ego)]

In [25]:
result = session.query(Person).filter(Person.ssn == 1)
result

In [26]:
for r in result:
    print(r)

(1) (Mike) (Smith)


In [28]:
result = session.query(Thing).all()
print(result)

[<__main__.Thing object at 0x000001C9E41936A0>, <__main__.Thing object at 0x000001C9E41931C0>]


### OTHER APPROACH

In [30]:
import sqlalchemy as db

In [46]:
engine = db.create_engine('sqlite:///test3.sqlite')
conn = engine.connect()
metadata = db.MetaData()

In [48]:
people = db.Table('people', metadata, autoload=True, autoload_with=engine)

In [53]:
people.columns.keys()

['ssn', 'first_name', 'last_name', 'gender', 'age']

In [58]:
query = people.select()
print(query)

SELECT people.ssn, people.first_name, people.last_name, people.gender, people.age 
FROM people


In [59]:
exe = conn.execute(query)
results = exe.fetchall()
results

[(1, 'Mike', 'Smith', 'm', 36),
 (2, 'Anna', 'Smith', 'm', 25),
 (3, 'Scott', 'Dylan', 'm', 18),
 (4, 'Cassie', 'Ego', 'm', 20)]